# Connect the entire loop

## Load Data Generator and PDPTW Environment

In [115]:
#cd rl4co\ git

NameError: name 'pwd' is not defined

In [5]:
import argparse
import os
from datetime import datetime
from pathlib import Path

import numpy as np
import torch
import pandas as pd

from ppo_agent import PPOAgent
from dvrp_env import DVRPEnv
from vectorized_env import VectorizedDVRPEnv
from embedding import (
    EmbeddingFFN, flexibility_personalities, n_flexibilities,
    update_embedding_model
)
from collections import deque
from meta_train import create_eval_envs

/opt/anaconda3/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the

In [17]:
eval_env, eval_initial_states = create_eval_envs(
    num_eval_envs=8,
    num_customers=30,
    eval_seed=9999,  # Different seed from training
    traveler_decisions_path="traveler_decisions_augmented.csv",
    device='cpu'
)

/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:210: Attribute 'policy' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['policy'])`.
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:210: Attribute 'policy' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['policy'])`.
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:210: Attribute 'policy' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['policy'])`.
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:210: Attribute 'policy' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended t

In [37]:
from vectorized_env import VectorizedDVRPEnv

num_envs = 4
num_customers = 5
decisions_path = "traveler_decisions_augmented.csv"

vec_env = VectorizedDVRPEnv(
    num_envs=num_envs,
    num_customers=num_customers,
    seed=42,
    traveler_decisions_path=decisions_path
)

states, infos = vec_env.reset()

/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:210: Attribute 'policy' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['policy'])`.
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:210: Attribute 'policy' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['policy'])`.
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:210: Attribute 'policy' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['policy'])`.
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:210: Attribute 'policy' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended t

In [49]:

import gymnasium as gym
import numpy as np
import pandas as pd
import torch
from gymnasium import spaces
from rl4co.envs.routing import SFGenerator, PDPTWEnv
from ortools_solver import darp_solver
from rl4co.models.zoo import AttentionModel, AttentionModelPolicy
from tensordict.tensordict import TensorDict

In [45]:
SLICE_FIELDS = {"h3_indices", "time_windows", "demand", "locs", "action_mask", "visited"}

In [71]:
env = PDPTWEnv()
env.reset(batch_size=[1])

TensorDict(
    fields={
        action_mask: Tensor(shape=torch.Size([1, 61]), device=cpu, dtype=torch.bool, is_shared=False),
        capacity: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, is_shared=False),
        current_node: Tensor(shape=torch.Size([1, 1]), device=cpu, dtype=torch.int64, is_shared=False),
        current_time: Tensor(shape=torch.Size([1, 1]), device=cpu, dtype=torch.float32, is_shared=False),
        demand: Tensor(shape=torch.Size([1, 61]), device=cpu, dtype=torch.float32, is_shared=False),
        done: Tensor(shape=torch.Size([1, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        flexibility: Tensor(shape=torch.Size([1, 61]), device=cpu, dtype=torch.float32, is_shared=False),
        h3_indices: Tensor(shape=torch.Size([1, 61]), device=cpu, dtype=torch.int64, is_shared=False),
        i: Tensor(shape=torch.Size([1, 1]), device=cpu, dtype=torch.int64, is_shared=False),
        locs: Tensor(shape=torch.Size([1, 61, 2]), device=cpu, dt